In [120]:
import time
import xlrd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import datetime
from datetime import datetime
from datetime import timedelta
import dateutil.relativedelta
import calendar
import re
import pandas
import pandas as pd

我手头的项目是要爬2010年到现在每个月/每一天的关于同居的微博。目前常见的微博爬虫有三个目标网站，一个是pc端口s.weibo.com，一个是智能手机端口m.weibo.cn，一个是非常老的手机端口weibo.cn。很多微博爬虫都采用智能手机端端口，因为可以爬到很多数据，只要不停往下滑应该就能爬完。但是我目前还没有发现智能手机端口如何搜索指定的日期或时间段，这就不得不用pc端口或者非常老的那个手机端口了。

非常老的手机端口的好处在于不用管验证码，直接就可以登陆，省去了非常多的麻烦。而且不存在‘展开全文’的问题，pc端和智能手机端需要展开原文。而且每一个日期可以搜索到100页，pc端只能搜索到50页。虽然可能pc端每一页比手机端口要多，但是应该还是这个端口能爬到的更多一点。还有一个原因是可以直接勾选原创微博，省去了看上去爬了很多，但实际上爬完了还需要删除所有的转发的过程（虽然复制粘贴的还是要额外删除）。

用selenium是因为微博会反爬虫，selenium可以模拟正常登陆的流程，而且一定要在每一次点击一个页面之后停顿几秒钟（sleep），不然很容易被封号甚至封IP。

In [164]:
days_normal = []
days_datetime = []
usernames = []
user_links = []
posts = []
likes = []
reblogs = []
comments = []
comment_links = []
post_date = []
post_time = []
data = {}

In [3]:
#我本来想按照月来爬的，但是输入指定月份例如20100101-20100131之后
#爬到的几乎都是1月31号的微博，这样的样本不是很好
#所以还是每天都爬2、3页当做样本，过几个星期我再每天都爬100页试试

def dates_in_year(YEAR):
    YEAR = int(YEAR)
    MONTH = 1
    DAY = 1
    date = datetime(YEAR,MONTH,DAY)
    
    days_in_year = 365 + (1*calendar.isleap(YEAR))
    for i in range(days_in_year):
        days_normal.append(date.strftime("%Y")+date.strftime("%m")+date.strftime("%d"))
        days_datetime.append(date) 
        date = date + dateutil.relativedelta.relativedelta(days=1)     

In [4]:
def login():   
    #登陆 Login
    driver.get('https://weibo.cn/pub/?vt=' )
   
    elem = driver.find_element_by_xpath("//*[@class='ut']/a[1]").click();
    time.sleep(1)
    
    print("开始自动登陆Loading")
    elem = driver.find_element_by_xpath("//*[@id='loginName']");
    elem.send_keys(username)
    elem = driver.find_element_by_xpath("//*[@id='loginPassword']");
    elem.send_keys(password)
    elem = driver.find_element_by_xpath("//*[@id='loginAction']");
    elem.send_keys(Keys.ENTER)  
    time.sleep(1)

In [215]:
#每爬完一天清理一次临时数据
#clear temp data after each day
def clean_temp_data():
    usernames.clear()
    user_links.clear()
    posts.clear()
    likes.clear()
    reblogs.clear()
    comments.clear()
    comment_links.clear()
    post_date.clear()
    post_time.clear()

In [212]:
#每天保存
#save each day
def save_data():
    data['Posts'] = posts
    data['User Names'] = usernames
    data['User Links'] = user_links
    data['Likes'] = likes
    data['Reblogs'] = reblogs
    data['Comments'] = comments
    data['Comment Links'] = comment_links
    data['Post Date'] = post_date
    data['Post Time'] = post_time

In [113]:
#翻页 Next Page
def click_next_page():
    NextPage = driver.find_element_by_xpath("//*[@class='pa']/form/div/a[1]")
            #driver.find_element_by_xpath("//a[contains(text(),'下页')]").click()
    NextPage.click()
    time.sleep(0.5)

In [226]:
#开始爬一页微博。注意elements是可以抓取多个，element只能抓一个
#Scraping one page. Notice elements not element. Elements can find multiple element

def scrape_one_page():
    #找到发微博的用户名和连接
    #Get username and href link
    elem = driver.find_elements_by_xpath("//*[@class='nk']")
    for post in elem:
        usernames.append(post.text)
        user_links.append(post.get_attribute('href'))

    #找到微博内容
    #Get post content
    elem = driver.find_elements_by_xpath("//*[@class='ctt']")
    for post in elem:
        posts.append(post.text)

    #点赞
    #likes
    likes_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'赞')]")
    for like in elem:
        likes_temp.append(like.text)
    for like in likes_temp:
        like = re.sub(r'\D', '', like) #去除掉‘赞’这个字只保留数字Delete 'likes' text
        likes.append(like)
    
    #转发评论
    #reblogs
    reblog_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'转发')]")
    for reblog in elem:
        reblog_temp.append(reblog.text)
    for reblog in reblog_temp:
        reblog = re.sub(r'\D', '', reblog) #保留数字keep the number
        reblogs.append(reblog)  

    #评论和评论的链接comment and links to comment section
    comment_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'评论')]")
    for comment in elem:
        comment_temp.append(comment.text)
        comment_links.append(comment.get_attribute('href'))#评论的链接links to comment
    for comment in comment_temp:
        comment = re.sub(r'\D', '', comment) #保留数字keep the number
        comments.append(comment)  

    #发布日期和时间
    elem = driver.find_elements_by_xpath("//*[@class='ct']")
    for time in elem:
        post_date.append(time.text.split(' ')[0])
        post_time.append(time.text.split(' ')[1])
        post_date_normal.append()

In [227]:
#爬一天内的每一页
#Scrape every page in a day
def scrape_everything_in_a_day(target_page_per_day):    
    
    #先爬一页再翻页
    #scrape a page first then next page
    scrape_one_page()
    time.sleep(0.5)
    
    current_page = []
    max_page = []
    
    #当前页数和最大页数
    #get current page and max page
    elem = driver.find_element_by_xpath("//*[@class='pa']")
    elem = elem.text
    elem = elem.split('/')
    current_page = elem[0]
    max_page = elem[1]
    current_page = re.sub(r'\D','',current_page)
    max_page = re.sub(r'\D','',max_page)
 
    #如果超过最大页码，目标页码变为最大页码
    if int(max_page) <target_page_per_day:
        target_page_per_day = max_page
        
    for page in range(int(target_page_per_day)):
        click_next_page()
        scrape_one_page()
        time.sleep(0.5)
        
        elem = driver.find_element_by_xpath("//*[@class='pa']")
        elem = elem.text
        elem = elem.split('/')
        current_page = elem[0]
        current_page = re.sub(r'\D','',current_page)
        
        #到最大页数的时候停止
        #stop when hitting max page
        if int(current_page) == int(max_page):
            #保存数据，再把临时的清空
            save_data()
            break

接下来是搜索前后和按照日期循环
Below are before and after search and reiterate through dates

In [228]:
def open_search_page_first_time():
    #点开搜索页面 Open search tap
    driver.find_element_by_xpath("//a[contains(text(),'搜索')]").click()
    driver.find_element_by_xpath("//a[contains(text(),'高级搜索')]").click()
    time.sleep(0.5)

def open_search_page_reiterate():
    #点开‘更多’ Open'more'
    driver.find_element_by_xpath("//a[contains(text(),'更多')]").click()
    time.sleep(0.5)
    
def keyword_input(key):
    #输入关键词 Input Keywords
    #我这里是"同居" In this case 'cohabitation'
    elem = driver.find_element_by_xpath("//*[@type='text']")
    elem.send_keys(key)
    
def date_input(day):
    #开始年月日 Start data e.g, 20200101
    elem = driver.find_element_by_xpath("//*[@name='starttime']")
    elem.clear()
    elem.send_keys(day)

    #结束年月日一样 End data same
    elem = driver.find_element_by_xpath("//*[@name='endtime']")
    elem.clear()
    elem.send_keys(day)

In [24]:
def search_first_time(key):
    #上面def的两个function
    open_search_page_first_time()
    keyword_input(key)
    
    #勾选原创 Click original posts
    driver.find_element_by_xpath("//*[@name='hasori']").click()
    
    date_input(days_normal)
    
    #搜索 Search
    driver.find_element_by_xpath("//*[@type='submit']").click()
    
def search_reiterate(day):
    open_search_page_reiterate()
    
    #爬第二天所以要把第一天删去
    days_normal = days_normal[:1]
    date_input(day)
    
    #搜索 Search
    driver.find_element_by_xpath("//*[@type='submit']").click()

def login
def dates_in_year(target_year)
def date_input()
def scrape_by_one_page()
def next_page_until(target_page_per_day)
def search_first_time(key,day)
def search_reiterate(day)
这些是目前定义过的比较重要的
These are important defs so far

In [222]:
clean_temp_data()
scrape_everything_in_a_day(4)

In [225]:
data = pd.DataFrame(data)
data

,Posts,User Names,User Links,Likes,Reblogs,Comments,Comment Links,Post Date,Post Time
0,:女人失恋以后，听到大家都普遍反映前男友新找的那个姑娘没自己好看，通常都会无比强烈地暗爽。 ...,小黑爱小白,https://weibo.cn/ryxh784533,0,0,0,https://weibo.cn/comment/1EgWd?uid=1261131600&...,2010-01-01,22:44:38
1,:《恋爱前规则》这部电影拍得一点也不像原著《和空姐同居的日子》那么精彩，这些网络小说改编成电...,肉包儿和猪猪的纪念,https://weibo.cn/comex,0,0,0,https://weibo.cn/comment/1EeGI?uid=1403893343&...,2010-01-01,22:18:39
2,:不论东西多少，只要是搬家都是大工程啊。我跟同居者聊天，打算在家里放一块匾，上书“弱智儿童欢...,小壹,https://weibo.cn/u/1657282170,0,0,2,https://weibo.cn/comment/1Edu3?uid=1657282170&...,2010-01-01,22:04:23
3,:看了恋爱前规则，对小说产生了兴趣，《和空姐同居的日子》找出来看，刚刚看完，无限yy,成熟孩子气,https://weibo.cn/lcj0431046,0,0,0,https://weibo.cn/comment/1EbVw?uid=1644651464&...,2010-01-01,21:45:27
4,:@claireg 同居也是一种选择。,阿力克狮,https://weibo.cn/amour,0,0,0,https://weibo.cn/comment/1EaV7?uid=1113519487&...,2010-01-01,21:33:31
5,:@thales 提倡合法同居。,阿力克狮,https://weibo.cn/amour,0,0,0,https://weibo.cn/comment/1EaJ4?uid=1113519487&...,2010-01-01,21:30:53
6,:@thales 你就忍心人家下半辈子活在坟墓之中？撕掉本本，还可以同居嘛。,阿力克狮,https://weibo.cn/amour,0,0,0,https://weibo.cn/comment/1Ear6?uid=1113519487&...,2010-01-01,21:26:55
7,:(1/2)知道周迅的越来越多，知道鲁迅的越来越少；知道马克的越来越多，知道马克思的越来越少...,i幸福i晓林,https://weibo.cn/chinalunyu,0,0,0,https://weibo.cn/comment/1DLOh?uid=1656366057&...,2010-01-01,14:27:57
8,:如果爱一个人，千万不要与他同居或是结婚。 维持一个辽阔的距离，偶遇，可以爱慕的目光致敬， ...,田田的牧歌,https://weibo.cn/u/1662934211,0,0,0,https://weibo.cn/comment/1DDuL?uid=1662934211&...,2010-01-01,11:53:24
9,:刚刚看完小说《和空姐同居的日子》。感觉非常好，我觉得虽然有点脱离实际，但不可否认的是它的故...,神无月琉璃,https://weibo.cn/u/1651816812,0,0,0,https://weibo.cn/comment/1DtU5?uid=1651816812&...,2010-01-01,08:53:49


In [110]:
def scrape_by_date(key, target_year, target_page_per_day):
    login()
    dates_in_year(target_year)
    search_first_time(key)
    for day in dates_normal:
        search_reiterate(key,day)
        for day in days_normal[:1]:
            clean_temp_data()
            scrape_everything_in_a_day(target_page_per_day)
            date_input_and_search(day)
            print('Finish' + day)

if __name__ == '__main__':
    username = "17064888409" #用户名username
    password = "rw297421" #密码password
    driver = webdriver.Chrome()#可以换成firefox.Can use firefox        
    scrape_by_date('同居',2010, 10)
#页数可以设置成max_page

开始自动登陆，若出现验证码手动验证.Loading


UnboundLocalError: local variable 'max_page' referenced before assignment

In [ ]:
def scrape_by_date(key, target_year, target_page_per_day):
    login()
    dates_in_year(target_year)
    search_first_time(key)
    for day in dates_normal:
        search_reiterate(key,day)
        for day in days_normal[:1]:
            scrape_one_page()
            next_page_until(target_page_per_day)
            date_input_and_search(day)
            print('Finish' + day)

if __name__ == '__main__':
    username = "17064888409" #用户名username
    password = "rw297421" #密码password
    driver = webdriver.Chrome()#可以换成firefox.Can use firefox        
    scrape_by_date('同居',2010, 10)
#页数可以设置成max_page

In [5]:
def scrape_by_date(key, target_year, target_page_per_day):
    login()
    dates_in_year(target_year)
    search_first_time(key)
    for day in dates_normal:
        search_reiterate(key,day)
        for day in days_normal[:1]:
            scrape_one_page()
            next_page_until(target_page_per_day)
            date_input_and_search(day)
            print('Finish' + day)

if __name__ == '__main__':
    username = "17064888409" #用户名username
    password = "rw297421" #密码password
    driver = webdriver.Chrome()#可以换成firefox.Can use firefox        
    scrape_by_date('同居',2010, 10)
#页数可以设置成max_page

开始自动登陆Loading
